# Proyecto (obtención dataset)

In [9]:
# --- General ---
import os
from datetime import datetime

# --- Data ---
import pandas as pd
from pandasql import sqldf
import numpy as np

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
# from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher


# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')
# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

## Inicialización peticiones Sophia2

In [10]:
# # --- Nombre archivo ---
# nombre_archivo = 'data'

# # --- Parameters queries ---
# # País
# country="chile"
# # Fecha
# from_="2010-01-01"
# to_="2010-12-31"
# media_outlets = ['elciudadano']
# nombre_archivo = nombre_archivo + '_' + from_ + '_' + to_ + '.csv'


# # --- Data Connection ---
# IP = "search.sophia2.org"
# PORT = 9200
# USER= "elastic"
# PASS= password

# # --- Connection ---
# es = elasticsearch.Elasticsearch(
#     IP,
#     http_auth=(USER, PASS)
# )

# # --- Queries ---
# query = { 
#     "bool": { 
#     "filter": [
#         {"range": {
#       "date": {
#         "gte": from_,
#         "lt": to_
#       }
#       }},

#         { "term":  { "country": country }},
#         { "terms":  { "media_outlet": media_outlets }} 
#     ]
#     }  
# }
# # --- Confirmación de conexión ---
# res = es.search(index="news", query=query, size=10000)
# print("Son %d noticias encontradas..." % res['hits']['total']['value'])

In [11]:
def automatico(fechas):
    # --- Data Connection ---
    IP = "search.sophia2.org"
    PORT = 9200
    USER= "elastic"
    PASS= 'dummy_elastic'

    # --- Connection ---
    es = elasticsearch.Elasticsearch(
        IP,
        http_auth=(USER, PASS)
    )
    
    for from_,to_ in (fechas):
        # --- Nombre archivo ---
        nombre_archivo = 'data'
        # --- Parameters queries ---
        # País
        country="chile"
        # Fecha
        media_outlets = ['elciudadano']
        nombre_archivo = nombre_archivo + '_' + from_ + '_' + to_ + '.csv'

        query = { 
            "bool": { 
            "filter": [
                {"range": {
            "date": {
                "gte": from_,
                "lt": to_
            }
            }},

                { "term":  { "country": country }},
                { "terms":  { "media_outlet": media_outlets }} 
            ]
            }  
        }
        res = es.search(index="news", query=query, size=10000)
        if (len(res) == 10000):
            return f'Revisar: {from_},{to_}'

        # --- Columnas Dataframe ---
        data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
        df = pd.DataFrame(data)  

        # --- Información Dataframe---
        for hit in res['hits']['hits']:

            id_news = hit['_source']['id_news']
            country = hit['_source']['country']
            media_outlet = hit['_source']['media_outlet']
            url = hit['_source']['url']
            title = hit['_source']['title']
            text = hit['_source']['text']
            date = hit['_source']['date']
            
            new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
            df = pd.concat([df, pd.DataFrame.from_records([new_row])])
            # df = df.append(new_row, ignore_index=True)


        # Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
        df = df.reset_index(drop=True)

        # --- Elimina resiuduos ---
        # Elimina duplicados
        df = df.drop_duplicates(subset='url', keep='first')
        # Elimina textos vacíos o con muy poco contenido
        df = df[df['text'].str.len() > 50]

        # Dejamos id_news,media_outlet,url,title,date
        df = df[['id_news','url','title','date']]

        # --- Info Archivo ---
        print("El archivo se llama:")
        print(nombre_archivo)

        # --- Guarda archivo ---
        df.to_csv("./data/"+nombre_archivo)


In [12]:
from_="2010-01-01"
to_="2010-06-31"
fechas = [
    ('2010-01-01','2010-06-30'),
    ('2010-06-30','2010-12-31'),
    ('2011-01-01','2011-06-30'),
    ('2011-06-30','2011-12-31'),
    ('2012-01-01','2012-06-30'),
    ('2012-06-30','2012-12-31'),
    ('2013-01-01','2013-06-30'),
    ('2013-06-30','2013-12-31'),
    ('2014-01-01','2014-06-30'),
    ('2014-06-30','2014-12-31'),
    ('2015-01-01','2015-06-30'),
    ('2015-06-30','2015-12-31'),
    ('2016-01-01','2016-06-30'),
    ('2016-06-30','2016-12-31'),
    ('2017-01-01','2017-06-30'),
    ('2017-06-30','2017-12-31'),
    ('2018-01-01','2018-06-30'),
    ('2018-06-30','2018-12-31'),
    ('2019-01-01','2019-06-30'),
    ('2019-06-30','2019-12-31'),
    ('2020-01-01','2020-06-30'),
    ('2020-06-30','2020-12-31'),
    ('2021-01-01','2021-06-30'),
    ('2021-06-30','2021-12-31')
]
automatico(fechas)

El archivo se llama:
data_2010-01-01_2010-06-30.csv
El archivo se llama:
data_2010-06-30_2010-12-31.csv
El archivo se llama:
data_2011-01-01_2011-06-30.csv
El archivo se llama:
data_2011-06-30_2011-12-31.csv
El archivo se llama:
data_2012-01-01_2012-06-30.csv
El archivo se llama:
data_2012-06-30_2012-12-31.csv
El archivo se llama:
data_2013-01-01_2013-06-30.csv
El archivo se llama:
data_2013-06-30_2013-12-31.csv
El archivo se llama:
data_2014-01-01_2014-06-30.csv
El archivo se llama:
data_2014-06-30_2014-12-31.csv
El archivo se llama:
data_2015-01-01_2015-06-30.csv
El archivo se llama:
data_2015-06-30_2015-12-31.csv
El archivo se llama:
data_2016-01-01_2016-06-30.csv
El archivo se llama:
data_2016-06-30_2016-12-31.csv
El archivo se llama:
data_2017-01-01_2017-06-30.csv
El archivo se llama:
data_2017-06-30_2017-12-31.csv
El archivo se llama:
data_2018-01-01_2018-06-30.csv
El archivo se llama:
data_2018-06-30_2018-12-31.csv
El archivo se llama:
data_2019-01-01_2019-06-30.csv
El archivo s

## Creación dataframe

In [13]:
# # --- Columnas Dataframe ---
# data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
# df = pd.DataFrame(data)  

# # --- Información Dataframe---
# for hit in res['hits']['hits']:

#     id_news = hit['_source']['id_news']
#     country = hit['_source']['country']
#     media_outlet = hit['_source']['media_outlet']
#     url = hit['_source']['url']
#     title = hit['_source']['title']
#     text = hit['_source']['text']
#     date = hit['_source']['date']
    
#     new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
#     df = pd.concat([df, pd.DataFrame.from_records([new_row])])
#     # df = df.append(new_row, ignore_index=True)


# # Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
# df = df.reset_index(drop=True)

# # --- Elimina resiuduos ---
# # Elimina duplicados
# df = df.drop_duplicates(subset='url', keep='first')
# # Elimina textos vacíos o con muy poco contenido
# df = df[df['text'].str.len() > 50]

In [14]:
# print(f"Son {len(df)} noticias encontradas que se utilizarán")
# df.head()

## Guardamos la información CSV

In [15]:
# # --- Info Archivo ---
# print("El archivo se llama:")
# print(nombre_archivo)

# # --- Guarda archivo ---
# df.to_csv("./data/"+nombre_archivo)

###  Con esto tenemos toda la información necesaria para cargar los datos y utilizarlos.